In [6]:
import pandas as pd
import experiment_util as util
import models
import numpy as np
from ucimlrepo import fetch_ucirepo
import random

np.random.seed(0)
random.seed(0)

In [71]:
diabetes = fetch_ucirepo(id=296) 

raw_df=pd.DataFrame(diabetes.data.original)

df=raw_df.copy()
df

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\.venv\Lib\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),NaN,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),NaN,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),NaN,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),NaN,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [72]:
df["readmitted"]=df["readmitted"].replace(['<30','>30'],['Yes','Yes'])
#unifiying <30 and >30 label to simplify the dataset task (as well as make different datasets comparable)
df=df.replace(['?'],[np.nan])
#Unifiying missing values
df=df.drop(df.loc[df["gender"]=="Unknown/Invalid",].index, axis=0)
#Remove rows where gender value are invalide
pre_df=util.preporcess_transform(df)

In [107]:
folds_index=np.array_split(df.sample(frac=1).index,5)
eval=util.evaluation(df,"gender","readmitted","Male",folds_index)
result=pd.DataFrame([],columns=util.column)
result.columns.name="Diabetes"
result

Diabetes,False Omissin Rate Disparity,False Discover Rate Disparity,Predicted Positive Ratio Disparity,Predicted Prevalance Disparity


In [108]:
result=util.append_Series(result,eval.cross_validation("Random Forest",models.RandomForest,pre_df))
result

c:\Users\15783\source\repos\Ethical-Framework\Aequitas\experiment_util.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df=pd.concat([dataframe,pd.DataFrame(series.to_list(),index=[series.name],columns=column)])


Diabetes,False Omissin Rate Disparity,False Discover Rate Disparity,Predicted Positive Ratio Disparity,Predicted Prevalance Disparity
Random Forest,1.088803,1.022523,1.256331,1.080573


In [109]:
result=util.append_Series(result,eval.cross_validation("Gradient Boost",models.GradientBoost,pre_df))
result

Diabetes,False Omissin Rate Disparity,False Discover Rate Disparity,Predicted Positive Ratio Disparity,Predicted Prevalance Disparity
Random Forest,1.088803,1.022523,1.256331,1.080573
Gradient Boost,1.051992,1.035081,1.222933,1.051835


In [ ]:
result=util.append_Series(result,eval.cross_validation("Gradient Boost",models.Fairgbm,pre_df))
result

In [ ]:
result=util.append_Series(result,eval.cross_validation("Gradient Boost",models.Fairlearn,pre_df))
result

In [ ]:
result.to_csv("diabetes_result.csv")

In [105]:
import importlib
importlib.reload(models)
importlib.reload(util)

<module 'experiment_util' from 'c:\\Users\\15783\\source\\repos\\Ethical-Framework\\Aequitas\\experiment_util.py'>